Agregar features segun el nombre de usuario <br>
Usar la prediccion de un predictor como feature para otro predictor <br>
Usar varios clasificadores y promediar los resultados

In [1]:
from collections import Counter

In [2]:
import gc
import numpy as np

In [3]:
gc.collect()

0

In [4]:
from sklearn.metrics import make_scorer

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [6]:
from sklearn.ensemble import BaggingRegressor

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import category_encoders

/home/javier/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/javier/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [8]:
import xgboost as xgb

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [10]:
import pandas as pd

In [11]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [12]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [13]:
processed = pd.read_csv('features_01.csv')

In [14]:
processed = processed.set_index('person')
processed.head()

mayor_evento              prod_visto      dispos  compras  \
person                                                                     
0008ed71           checkout                 Ninguno    Computer        0   
00091926     viewed product               iPhone 6S    Computer        0   
00091a7a      brand listing               iPhone 6S  Smartphone        0   
000ba417     viewed product  Samsung Galaxy A3 Duos    Computer        1   
000c79fe  searched products                iPhone 7  Smartphone        0   

          visitas  cant_prod_vistos        primer_visita        ultima_visita  \
person                                                                          
0008ed71        2                 0  2018-05-17 12:27:47  2018-05-17 16:28:37   
00091926       34               372  2018-05-03 22:08:29  2018-05-31 19:52:03   
00091a7a        1                 3  2018-03-26 14:51:11  2018-03-26 14:56:58   
000ba417        6               153  2018-05-17 11:11:45  2018-05-26 13:09:22   
000c79fe        1                 3  2018-05-29 00:27:47  2018-05-29 00:38:07   

          cant_eventos  permanencia  hace_cuanto_visito  eventos_por_dia  \
person                                                                     
0008ed71             6            1                  14              6.0   
00091926           448           28                   0             16.0   
00091a7a            10            1                  66             10.0   
000ba417           206           10                   5             20.6   
000c79fe            17            1                   2             17.0   

           channel  visitas_por_dia  
person                               
0008ed71  Referral         2.000000  
00091926    Direct         1.214286  
00091a7a      Paid         1.000000  
000ba417    Direct         0.600000  
000c79fe      Paid         1.000000

In [15]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [16]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

/home/javier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
training_features.head()

mayor_evento                prod_visto      dispos  compras  \
person                                                                       
0008ed71           checkout                   Ninguno    Computer        0   
000c79fe  searched products                  iPhone 7  Smartphone        0   
001802e4    ad campaign hit                 iPhone 6S  Smartphone        0   
0019e639     viewed product  Motorola Moto G2 3G Dual    Computer        2   
001b0bf9           checkout    Samsung Galaxy S8 Plus     Ninguno        1   

          visitas  cant_prod_vistos  cant_eventos  permanencia  \
person                                                           
0008ed71        2                 0             6            1   
000c79fe        1                 3            17            1   
001802e4        1                 4            19            1   
0019e639       19               189           471          115   
001b0bf9        0                 2             7            1   

          hace_cuanto_visito  eventos_por_dia   channel  visitas_por_dia  
person                                                                    
0008ed71                  14         6.000000  Referral         2.000000  
000c79fe                   2        17.000000      Paid         1.000000  
001802e4                   0        19.000000      Paid         1.000000  
0019e639                   9         4.095652      Paid         0.165217  
001b0bf9                   9         7.000000   Ninguno         0.000000

In [18]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [19]:
encoded_training.head()

0         1         2   3   4    5    6    7   8          9   \
0  0.007335  0.042654  0.069914   0   2    0    6    1  14   6.000000   
1  0.053592  0.049793  0.034791   0   1    3   17    1   2  17.000000   
2  0.018508  0.034037  0.034791   0   1    4   19    1   0  19.000000   
3  0.055779  0.026954  0.069914   2  19  189  471  115   9   4.095652   
4  0.007335  0.054054  0.017361   1   0    2    7    1   9   7.000000   

         10        11  
0  0.065251  2.000000  
1  0.042291  1.000000  
2  0.042291  1.000000  
3  0.042291  0.165217  
4  0.017361  0.000000

In [21]:
forests = RandomForestRegressor()
forests.fit(encoded_training,training_labels)
forests.feature_importances_

array([0.03853501, 0.13792589, 0.02264632, 0.02925079, 0.05398191,
       0.13352286, 0.12514243, 0.06830306, 0.1269321 , 0.13660073,
       0.04570558, 0.08145331])

In [22]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10],
    'max_samples':[1.0],
    'max_features':[1.0], 
}
grid = GridSearchCV(BaggingRegressor(),params,scoring=make_scorer(roc_auc_score),cv=2,verbose=0)

In [23]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [24]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise',
       estimator=BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_samples': [1.0], 'max_features': [1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=0)

In [25]:
grid.score(X=xte,y=yte)

0.6775036078513271

In [26]:
xgbc= grid.best_estimator_

In [27]:
xgbc.fit(xtr,ytr)
p=xgbc.predict(xte)
roc_auc_score(y_score=p,y_true=yte)

0.646410935381754

In [28]:
xgbc.fit(encoded_training,training_labels)

BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [29]:
prediction_submit = xgbc.predict(encoded_predict)

In [30]:
predict_features['label'] = prediction_submit

In [31]:
predict_features['label'].value_counts()

0.000000    13216
0.100000     3406
0.200000     1447
0.300000      683
0.400000      308
0.500000      154
0.600000       67
0.700000       37
0.800000       24
0.053803       13
0.900000       12
0.050000       12
0.166667        3
0.080000        3
0.116667        3
0.350000        2
0.216667        2
0.022500        2
0.138690        2
0.233333        2
0.150000        2
0.023611        2
0.012500        2
0.025000        2
0.042857        1
0.033333        1
0.016667        1
0.120000        1
0.305000        1
0.125000        1
0.250000        1
0.175000        1
0.673333        1
Name: label, dtype: int64

In [32]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [33]:
subm = subm.set_index('person')

In [34]:
#subm.to_csv('submit.csv')